In [1]:
print("ok")

ok


In [1]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain



In [2]:
%pwd

'c:\\Users\\PUJA KUMARI\\Desktop\\SourceCodeAnalysis\\research'

In [3]:
!mkdir text_repo

In [4]:
#clone the repo inside text_repo
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/Pujakumari1202/audio_transcript.git", to_path=repo_path)




In [8]:
# load the files (extract all the python file)  
# give the loader object
loader=GenericLoader.from_filesystem(repo_path,
                                      glob="**/*",
                                      suffixes=[".py"],
                                      parser=LanguageParser(language=Language.PYTHON,parser_threshold=500)
)

In [9]:
# now load the  documents
documents=loader.load()

In [10]:
documents

[Document(page_content='import openai\nimport os\nfrom dotenv import load_dotenv\nfrom flask import Flask, request, jsonify, render_template\n\n\n# load the api key\nload_dotenv()\nOPENAI_API_KEY=os.getenv("OPENAI_API_KEY")\n\n#add this openai key inside my openai key\nopenai.api_key=OPENAI_API_KEY\n\n\n#initialize the flask\napp = Flask(__name__)\n\n\n# audio will be upload in static folder\napp.config["UPLOAD_FOLDER"]="static"\n\n\n#create the route look for 2 kinds of req get and post\n@app.route("/", methods=["GET","POST"])\n\n#if this match then it will execute this main function\ndef main():\n    # if the req is post then\n    if request.method =="POST":\n        #take the language input \n        language= request.form["language"]\n\n        #get the file\n        file= request.files["file"]\n\n        # if file is present or not check\n        if file:  # save in static folder\n            filename=file.filename   #give the file name\n            file.save(os.path.join(app.conf

In [11]:
# now see the length of the documents
len(documents)

2

In [12]:
documents[0]

Document(page_content='import openai\nimport os\nfrom dotenv import load_dotenv\nfrom flask import Flask, request, jsonify, render_template\n\n\n# load the api key\nload_dotenv()\nOPENAI_API_KEY=os.getenv("OPENAI_API_KEY")\n\n#add this openai key inside my openai key\nopenai.api_key=OPENAI_API_KEY\n\n\n#initialize the flask\napp = Flask(__name__)\n\n\n# audio will be upload in static folder\napp.config["UPLOAD_FOLDER"]="static"\n\n\n#create the route look for 2 kinds of req get and post\n@app.route("/", methods=["GET","POST"])\n\n#if this match then it will execute this main function\ndef main():\n    # if the req is post then\n    if request.method =="POST":\n        #take the language input \n        language= request.form["language"]\n\n        #get the file\n        file= request.files["file"]\n\n        # if file is present or not check\n        if file:  # save in static folder\n            filename=file.filename   #give the file name\n            file.save(os.path.join(app.confi

In [13]:
# create the chunks  
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [14]:
# apply on the entire documents 
texts=documents_splitter.split_documents(documents)

In [15]:
#entire chunks
texts

[Document(page_content='import openai\nimport os\nfrom dotenv import load_dotenv\nfrom flask import Flask, request, jsonify, render_template\n\n\n# load the api key\nload_dotenv()\nOPENAI_API_KEY=os.getenv("OPENAI_API_KEY")\n\n#add this openai key inside my openai key\nopenai.api_key=OPENAI_API_KEY\n\n\n#initialize the flask\napp = Flask(__name__)\n\n\n# audio will be upload in static folder\napp.config["UPLOAD_FOLDER"]="static"\n\n\n#create the route look for 2 kinds of req get and post\n@app.route("/", methods=["GET","POST"])', metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='#if this match then it will execute this main function', metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='def main():\n    # if the req is post then\n    if request.method =="POST":\n        #take the language input \n        language= request.form["language"]\n\n        #get the file\n        fi

In [16]:
len(texts)

8

In [17]:
# now we need embedding model for that set openai_api key
# load the api key
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [18]:
# set the api key as our environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [19]:
# Initialized the embedding model
embeddings=OpenAIEmbeddings(disallowed_special=())


In [21]:
# Now initialised our chromedb
vectordb=Chroma.from_documents(texts,embedding=embeddings,persist_directory='./db')

In [22]:
# activate the vectordb
vectordb.persist()

In [23]:
# Initialized our llm model we can use any llm model
# llm=ChatOpenAI(model_name="gpt-4")
llm=ChatOpenAI()

In [24]:
#creating the memory
memory=ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [25]:
# creating our final chain
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)


In [26]:
question ="What is  jsonify function in flask?"

In [28]:
# pass this question inside qa
result=qa(question)

#Extract the answer from the result
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 16, updating n_results = 16


The `jsonify` function in Flask serializes data into JSON format and wraps it in a Response object. This function is commonly used to return JSON responses from Flask routes.


In [29]:
question ="What is  load_dotenv function?"

In [30]:
# pass this question inside qa
result=qa(question)

#Extract the answer from the result
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 16, updating n_results = 16


The `load_dotenv` function is used to load environmental variables from a .env file into the environment. It reads the key-value pairs from the .env file and sets them as environment variables, making them accessible to the application.
